In [1]:
import pandas as pd
import time
import numpy as np
from urllib.error import HTTPError

## Datasheet creation

First we used the API of Stemspy to get all games and there information. This was done using the ```pd.read_json()``` while using the API url as input. Because we didn't know how many games there where, we tried the api call until it returns a HTTPError. 

In [66]:
url = "https://steamspy.com/api.php?request=all&page="

ls = []
for i in range(80):
  try:
    data = pd.read_json(url+f"{i}").T
    ls.append(data)
    time.sleep(60)
  except HTTPError:
    break
df = pd.concat(ls, axis=0, ignore_index=True)

After generating the first part of our dataset, we saved it using the pandas function ```to_pickle()```. 

In [68]:
df.to_pickle("dataset.pkl")

After saving the first part, we had to get the detailed information for every instance. This was also done with the Steamspy API. 

In [2]:
df = pd.read_pickle("dataset.pkl")

detailed_url="https://steamspy.com/api.php?request=appdetails&appid="

To achieve that, we iterrate over the datset of part one and request the detailed information. Because of some default formating of ```pd.read_json()``` we had to construct a new pandas DataFrame to match our expectations. 

In [58]:
i = 0
for idx, rows in df.iterrows():
  temp_df = pd.read_json(detailed_url+f"{rows['appid']}")
  if temp_df.empty:
    continue
  detailed_df = pd.DataFrame([[temp_df.index.values],[temp_df.genre.values],[temp_df.languages.values]], 
             columns=[temp_df.appid[0]], 
             index=["tags","genre","language"]).T
  ls.append(detailed_df)
  time.sleep(1)
  i = i+1
  if i >= 20:
     df_detailed = pd.concat(ls, axis=0, ignore_index=False)
     df_detailed.to_pickle("detailed.pkl")
     i = 0

df_detailed = pd.concat(ls, axis=0, ignore_index=False)

After that we only need to combine the two parts into one dataset and save it to disk. 

In [4]:
df_detailed = pd.read_pickle("detailed.pkl")
print(df_detailed.shape)
df_detailed = df_detailed[~df_detailed.index.duplicated(keep='first')]

df_complete = pd.merge(df.set_index('appid'),df_detailed, left_index=True, right_index=True)

df_complete.to_pickle("complete_df_steam.pkl")

(74700, 4)


/home/florian/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [5]:
df_complete.sample(20, random_state=1)

,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,0,tags,genre,language
882910,BATTLE X,"Naviworks Co.,Ltd.","Naviworks Co.,Ltd.",,9,10,0,"0 .. 20,000",0,0,0,0,2500,2500,0,0,NaN,"[Action, Adventure, Massively Multiplayer, Sim...","[Action, Adventure, Massively Multiplayer, Sim...","[English, Korean, Simplified Chinese, English,..."
741670,Easy Red,Marco Amadei,Corvostudio di Amadei Marco,,767,186,0,"50,000 .. 100,000",219,0,226,0,399,399,0,2,NaN,"[Action, FPS, First-Person, Historical, Indie,...","[Action, Indie, Simulation, Strategy, Action, ...","[English, Italian, Simplified Chinese, Traditi..."
1897510,LAETUS.,"AjSpeed, Filip Budd",AjSpeed,,1,0,0,"0 .. 20,000",0,0,0,0,299,299,0,0,NaN,"[2D, 2D Platformer, Action, Colorful, Difficul...","[Action, Indie, Action, Indie, Action, Indie, ...","[English, French, Italian, German, Spanish - S..."
661280,Dresden Files Cooperative Card Game,Hidden Achievement LLC,Hidden Achievement LLC,,85,20,0,"0 .. 20,000",0,0,0,0,509,1699,70,4,NaN,"[2D, Board Game, Card Game, Casual, Co-op, Co-...","[Casual, Indie, Strategy, Casual, Indie, Strat...","[English, French, German, Russian, English, Fr..."
1080040,Moo Lander,The Sixth Hammer,The Sixth Hammer,,32,1,0,"0 .. 20,000",0,0,0,0,1899,1899,0,0,NaN,"[2D, 2D Platformer, 4 Player Local, Action, Ad...","[Action, Adventure, Indie, Action, Adventure, ...","[English, Italian, German, Spanish - Spain, En..."
517620,Gooblins,"Deform Studio, LLC","Waveform Logic, Inc",,9,2,0,"0 .. 20,000",0,0,0,0,399,399,0,0,NaN,"[Action, Casual, FPS, Indie, Shooter, VR]","[Action, Casual, Indie, Action, Casual, Indie,...","[English, English, English, English, English, ..."
411620,Pane In The Glass,Rewind:VR,Rewind,,21,13,0,"0 .. 20,000",0,0,0,0,499,499,0,0,NaN,"[Action, Casual, Simulation, VR]","[Action, Casual, Simulation, Action, Casual, S...","[English, English, English, English]"
1213860,Cut Smash Wrap,IndigoWare,IndigoWare,,12,1,0,"0 .. 20,000",0,0,0,0,199,199,0,0,NaN,"[2D, Addictive, Arcade, Cartoony, Casual, Cute...","[Casual, Indie, Strategy, Casual, Indie, Strat...","[English, Portuguese - Brazil, English, Portug..."
1046130,Tiki Trials,Last Minute Entertainment,Last Minute Entertainment,,14,0,0,"0 .. 20,000",0,0,0,0,0,0,0,0,NaN,"[2.5D, Action, Fast-Paced, Indie, Platformer]","[Action, Indie, Action, Indie, Action, Indie, ...","[English, English, English, English, English]"
925600,Afterlifes,Nockdev Entertainment,Nockdev Entertainment,,1,2,0,"0 .. 20,000",0,0,0,0,1799,1799,0,0,NaN,"[Action, Adventure, Early Access, Indie, Violent]","[Action, Adventure, Indie, Early Access, Actio...","[English, English, English, English, English]"
